## DIA 139: Convertir el Envío de Reportes en un Microservicio con FastAPI 

Hoy aprenderás a encapsular el proceso de generación y envío de reportes en PDF por usuario dentro de una API REST usando FastAPI, lo que permite:

🧩 Llamar al envío de reportes desde otros sistemas

🖥️ Exponer una API para enviar reportes bajo demanda

🤖 Automatizar tareas desde cron, bots, o frontend JavaScript

🧩 1. Instalar FastAPI y Uvicorn
bash
Copiar
Editar
pip install fastapi uvicorn
🧩 2. Estructura del microservicio
bash
Copiar
Editar
reportes_api/
├── main.py               ← Microservicio FastAPI
├── report_utils.py       ← Generación y envío de PDF
├── models.py             ← (opcional) Modelos como User
🧩 3. report_utils.py — lógica reutilizable
Aquí reutilizamos el código de los días anteriores:

python
Copiar
Editar
from io import BytesIO
from matplotlib import pyplot as plt
import pandas as pd
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from email.message import EmailMessage
import smtplib

def crear_pdf_desde_dataframe(df):
    buffer = BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=A4)
    styles = getSampleStyleSheet()
    story = []

    story.append(Paragraph("📄 Informe de Reportes", styles['Title']))
    story.append(Spacer(1, 12))
    story.append(Paragraph(f"Total reportes: {len(df)}", styles['Normal']))
    story.append(Spacer(1, 12))

    # Gráfico por categoría
    fig, ax = plt.subplots(figsize=(6, 3))
    df['Categoría'].value_counts().plot(kind='bar', ax=ax, color='green')
    ax.set_title('Por Categoría')
    img = BytesIO()
    plt.tight_layout()
    fig.savefig(img, format='png')
    img.seek(0)
    plt.close(fig)

    story.append(Image(img, width=400, height=200))
    doc.build(story)
    buffer.seek(0)
    return buffer

def enviar_pdf_por_correo(destinatario, pdf_buffer):
    remitente = 'tucorreo@gmail.com'
    clave = 'tu_contraseña_app'

    msg = EmailMessage()
    msg['Subject'] = '📄 Tu reporte personalizado'
    msg['From'] = remitente
    msg['To'] = destinatario
    msg.set_content('Aquí está tu reporte en PDF.')

    pdf_buffer.seek(0)
    msg.add_attachment(pdf_buffer.read(), maintype='application', subtype='pdf', filename='reporte.pdf')

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(remitente, clave)
        smtp.send_message(msg)
🧩 4. main.py — microservicio con FastAPI
python
Copiar
Editar
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from report_utils import crear_pdf_desde_dataframe, enviar_pdf_por_correo
import pandas as pd

app = FastAPI()

class ReporteRequest(BaseModel):
    email: str
    data: list  # Lista de diccionarios [{fecha, tipo, categoria}, ...]

@app.post("/enviar-reporte/")
def enviar_reporte(data: ReporteRequest):
    try:
        df = pd.DataFrame(data.data)
        if df.empty:
            raise HTTPException(status_code=400, detail="No se proporcionaron datos.")

        pdf = crear_pdf_desde_dataframe(df)
        enviar_pdf_por_correo(data.email, pdf)
        return {"status": "enviado", "email": data.email}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
🧪 5. Probar desde cURL o Postman
bash
Copiar
Editar
curl -X POST http://localhost:8000/enviar-reporte/ \
-H "Content-Type: application/json" \
-d '{"email":"usuario@correo.com", "data":[
      {"Fecha":"2024-01-01", "Tipo":"PDF", "Categoría":"Mensual"},
      {"Fecha":"2024-01-02", "Tipo":"CSV", "Categoría":"Diario"}
    ]}'
🧩 6. Ejecutar el microservicio
bash
Copiar
Editar
uvicorn main:app --reload